### Value at Risk (VaR) Analysis using the Parametric Method

Also known as the variance-covariance method. The parametric method estimates VaR by assuming that portfolio returns follow a normal distribution.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
adj_close_df = pd.read_csv('data/stocks.csv', parse_dates=['Date'], index_col='Date')

In [ ]:
tickers = ['SPY', 'BND', 'GLD', 'QQQ', 'VTI']

In [ ]:
log_returns_df = np.log(1 + adj_close_df.pct_change())
log_returns_df.dropna(inplace=True)

In [ ]:
log_returns_df.head()

###  Equally Weighted Portfolio
Equally weighted portfolio with a total value of 1,000,000

In [ ]:
portfolio_value = 1000000
weights = np.array([1/len(tickers)]*len(tickers))
print(weights)

In [ ]:
historical_returns_df = (log_returns_df * weights).sum(axis=1)

print(historical_returns_df.head())

### X-Day Historical Returns

X-day historical returns. In this example, we’ll use a 50-day window.

In [ ]:
days = 5
historical_x_day_returns = historical_returns_df.rolling(window=days).sum()
print(historical_x_day_returns)

In [ ]:
# corr = log_returns_df.corr()
# corr.style.background_gradient(cmap='coolwarm').set_precision(2)

### Create Covariance Matrix and Calculate Portfolio Standard Deviation

In [22]:

cov_matrix = log_returns_df.cov() * 252
print(cov_matrix)


          SPY       BND       GLD       QQQ       VTI
SPY  0.029468 -0.000011  0.001648  0.032584  0.030086
BND -0.000011  0.002322  0.002230  0.000207  0.000035
GLD  0.001648  0.002230  0.024443  0.001902  0.001799
QQQ  0.032584  0.000207  0.001902  0.041807  0.033284
VTI  0.030086  0.000035  0.001799  0.033284  0.030964


### Calculating Portfolio Standard Deviation

In [26]:
portfolio_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)
portfolio_std_dev

0.11602286357061328